In [ ]:
pip install -r requirements.txt

In [34]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Reshape
from tensorflow.keras.regularizers import L1L2

In [35]:
(X_treinamento, _), (_, _) = mnist.load_data()

In [36]:
X_treinamento = X_treinamento.astype('float32') / 255

In [37]:
X_treinamento = tf.data.Dataset.from_tensor_slices(X_treinamento).shuffle(buffer_size=600000).batch(batch_size=256)

In [ ]:
#Gerador
#1--> 500 --> 500

gerador = Sequential()
gerador.add(Dense(units = 500, input_dim=100, activation='relu', kernel_regularizer=L1L2(1e-5, 1e-5)))
gerador.add(Dense(units = 500, input_dim=100, activation='relu', kernel_regularizer=L1L2(1e-5, 1e-5)))
gerador.add(Dense(units = 784, activation='relu', kernel_regularizer=L1L2(1e-5, 1e-5)))
gerador.add(Reshape((28, 28)))
gerador.summary()

In [ ]:
discriminador = Sequential()
discriminador.add(InputLayer(input_shape=(28,28)))
discriminador.add(Flatten())
discriminador.add(Dense(units=500, activation='relu', kernel_regularizer=L1L2(1e-5, 1e-5)))
discriminador.add(Dense(units=500, activation='relu', kernel_regularizer=L1L2(1e-5, 1e-5)))
discriminador.add(Dense(units=1, activation='sigmoid', kernel_regularizer=L1L2(1e-5, 1e-5)))
discriminador.summary()

In [40]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()

In [41]:
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

In [42]:
def discriminator_loss(real_output, fake_output):
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  return total_loss

In [43]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [44]:
epochs = 100
noise_dim = 100

In [45]:
def train_step(images):
  noise = tf.random.normal([256, noise_dim])

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = gerador(noise, training = True)

    real_output = discriminador(images, training = True)
    fake_output = discriminador(generated_images, training = True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)

  gradients_of_generator = gen_tape.gradient(gen_loss, gerador.trainable_variables)
  gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminador.trainable_variables)

  generator_optimizer.apply_gradients(zip(gradients_of_generator, gerador.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminador.trainable_variables))

  return gen_loss, disc_loss

In [ ]:
for epoch in range(epochs):
  for image_batch in X_treinamento:
    gen_loss_batch, disc_loss_batch = train_step(image_batch)
  print(f'Época {epoch} | gen_loss: {gen_loss_batch} disc_loss {disc_loss_batch}')

In [ ]:
amostras = np.random.normal(size=(20, 100))
previsao = gerador.predict(amostras)
for i in range(previsao.shape[0]):
  plt.imshow(previsao[i, :], cmap='gray')
  plt.show()